In [ ]:
n_users = 5
n_items = 5

In [3]:
# Make up some random explicit feedback ratings
# and convert to a numpy array
import numpy as np
from scipy.sparse import rand as sprand

from mf import *

ratings = sprand(n_users, n_items, density=0.2, format='csr')
ratings.data = np.ndarray()#(np.random.randint(1, 5, size=ratings.nnz).astype(np.float64))

In [4]:
print(ratings.nnz)
ratings = ratings.toarray()

10


In [5]:
print(ratings)

[[ 2.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 3.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  2.  1.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  4.  0.  0.  2.  0.  0.]]


In [6]:
model = MatrixFactorization(n_users, n_items, n_factors=20)

In [7]:
loss_func = torch.nn.MSELoss()

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6) # learning rate

In [9]:
# Sort our data
from torch.autograd import Variable

rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

for row, col in zip(*(rows, cols)):
    # Turn data into variables
    rating: Variable = Variable(torch.FloatTensor([ratings[row, col]]))
    row = Variable(torch.LongTensor([np.long(row)]))
    col = Variable(torch.LongTensor([np.long(col)]))

    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)

    # Backpropagate
    loss.backward()

    # Update the parameters
    optimizer.step()
    print(row, col, loss)

tensor([3]) tensor([2]) tensor(91.0700, grad_fn=<MseLossBackward>)
tensor([3]) tensor([5]) tensor(0.3169, grad_fn=<MseLossBackward>)
tensor([3]) tensor([3]) tensor(86.5501, grad_fn=<MseLossBackward>)
tensor([1]) tensor([5]) tensor(15.3929, grad_fn=<MseLossBackward>)
tensor([4]) tensor([7]) tensor(54.2410, grad_fn=<MseLossBackward>)
tensor([4]) tensor([4]) tensor(149.3303, grad_fn=<MseLossBackward>)
tensor([3]) tensor([1]) tensor(5.0738, grad_fn=<MseLossBackward>)
tensor([0]) tensor([0]) tensor(47.1975, grad_fn=<MseLossBackward>)
tensor([0]) tensor([5]) tensor(11.3788, grad_fn=<MseLossBackward>)
tensor([2]) tensor([0]) tensor(30.9720, grad_fn=<MseLossBackward>)


In [11]:
#https://cs230-stanford.github.io/pytorch-getting-started.html
import utils
state = {'epoch': epoch + 1,
         'state_dict': model.state_dict(),
         'optim_dict' : optimizer.state_dict()}
utils.save_checkpoint(state,
                      is_best=is_best,      # True if this is the model with best metrics
                      checkpoint=model_dir) # path to folder

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not int

In [ ]:
#reg_loss_func = torch.optim.SGD(model.parameters(), lr=1e-6, weight_decay=1e-5)
# #adagrad_loss = torch.optim.Adagrad(model.parameters(), lr=1e-6)